# Import statements

Also includes a bit to suppress warnings about large images

In [1]:
import os, time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from astropy.io import fits
import requests, zipfile
import ipywidgets as w
#from IPython.display import display

#ignore image size warnings
import warnings
#warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", message="Image size")

# Interactive Methods

Starts the GUI-ish interface for each program

In [2]:
def start_rescale():
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    scale = w.FloatText(value=10., Text='scale')
    w.interact(rescale, scale=scale, name='lestermadeline_BW.jpg', width=width, height=height)
    return

def start_rebin():
    #bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact(rebin, name='lestermadeline_BW_5mile.jpg', bins=bins)
    return

def start_rescale_rebin():
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    scale = w.FloatText(value=10., Text='scale')
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact(rescale_rebin, scale=scale, bins=bins, name='lestermadeline_BW.jpg', width=width, height=height)
    return
    
def start_splitRGB():
    w.interact_manual(splitRGB, name='rainbowvalley.jpg', flip=w.fixed(True))
    return

def download():
    w.interact_manual(downloader, url='', name='file.jpg')
    return

def start_rebin_RGB():
    #bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact(rebin_RGB, name='Disneyland.png', bins=bins)
    return

def start_rescale_rebin_RGB():
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    scale = w.FloatText(value=10., Text='scale')
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact(rescale_rebin_RGB, scale=scale, bins=bins, name='Disneyland.png', width=width, height=height)
    return

And some manual methods (in case things are too slow)

In [3]:
def start_rescale_manual():
    #v = interact_manual(open_image,image_name="lestermadeline_BW.jpg", image_width=2600, image_height=1600)
    #display(v)
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    w.interact_manual(open_rescale, name='lestermadeline_BW.jpg', width=width, height=height)
    return

def open_rescale(name, width, height):
    #img_width = float(image_width)
    #img_height = float(image_height)
    scale = w.FloatText(value=10., Text='scale')
    w.interact_manual(rescale, scale=scale, image_name=w.fixed(name), 
                      image_width=w.fixed(width), image_height=w.fixed(height))
    return

def start_rebin_manual():
    #bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact_manual(rebin, name='lestermadeline_BW_5mile.jpg', bins=bins)
    return

def start_rescale_rebin_manual():
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    w.interact_manual(open_rescale_rebin, name='lestermadeline_BW.jpg', width=width, height=height)
    return 

def open_rescale_rebin(name, width, height):
    scale = w.FloatText(value=10., Text='scale')
    bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    w.interact_manual(rescale_rebin, scale=scale, bins=bins, img_name=w.fixed(name), 
                      img_width=w.fixed(width), img_height=w.fixed(height))
    return

def start_rebin_RGB_manual():
    #bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    bins = w.BoundedIntText(value=10, min=1, max=256, step=1,description='bins')
    w.interact_manual(rebin_RGB, name='Disneyland.png', bins=bins)
    return

def start_rescale_rebin_RGB_manual():
    width = w.FloatText(value=2600., Text='width')
    height = w.FloatText(value=1600., Text='height')
    w.interact_manual(open_rescale_rebin_RGB, name='lestermadeline_BW.jpg', width=width, height=height)
    return 

def open_rescale_rebin_RGB(name, width, height):
    scale = w.FloatText(value=10., Text='scale')
    bins = w.IntSlider(value=10, min=1, max=256, step=1,description='bins', continuous_update=False)
    w.interact_manual(rescale_rebin_RGB, scale=scale, bins=bins, img_name=w.fixed(name), 
                      img_width=w.fixed(width), img_height=w.fixed(height))

# Rescaling functions

In [4]:
def rescale_img(img, newscale, img_width, img_height):
    px_width, px_height = img.size
    widthscale = 1.0*img_width/px_width
    heightscale = 1.0*img_height/px_height
    
    if newscale < widthscale or newscale < heightscale:
        print("Warning: Your desired image scale is a finer resolution than in the original image; defaulting to the finest scale available")
        newscale = max(widthscale, heightscale)
    if newscale > img_width or newscale > img_height:
        print("Warning: Your desired image scale is larger than the original image; defaulting to the image size")
        newscale = min(img_width, img_height)
          
    new_px_width = int(1.0*px_width/(1.0*newscale/widthscale))
    new_px_height = int(1.0*px_height/(1.0*newscale/heightscale))
    #print(new_px_width, new_px_height)
    return img.resize((new_px_width, new_px_height))

#scale and img width/height have to be in same distance units
#in this case, miles
def rescale(scale, name="lestermadeline_BW.jpg", width=2600, height=1600):
    img = Image.open(name)
     
    %matplotlib notebook
    fig = plt.figure()
    
    new_img = rescale_img(img, scale, width, height)
    imshow(np.asarray(new_img), cmap='gray')
    plt.axis('off')
    
    plt.show()
    img.close()
    return 

# Rebinning methods (greyscale)

In [5]:
#function to help with rebinning a grey scale image
#def rebin_func(value, imgscale, truescale):
#    return truescale[np.where(imgscale <= value)[0][-1]]

#function takes an opened image to rebin
def rebin_img(img, nbins):
    bands = img.getbands()
    if len(bands) > 1: img = img.convert('L')
    
    imgarray = np.asarray(img)
    
    if nbins < 0:
        print("Warning: the number of bins has to be greater than 0; defaulting to 1 bin")
        nbins = 0
    if nbins > 256:
        print("Warning: maximum number of bins available is 256")
        
    imgmin = np.min(imgarray)
    imgmax = np.max(imgarray)
    imgscale = np.linspace(imgmin, imgmax, nbins+1)
    imgscale[-1] = imgscale[-1] + 1 #to capture max value
    truescale = np.linspace(0, 255, nbins)
    newimg = truescale[0]*np.ma.masked_where(np.logical_and(imgarray >= imgscale[0], imgarray < imgscale[1]), imgarray, copy=False).mask
    for i in range(1, nbins): 
        newimg = newimg+truescale[i]*np.ma.masked_where(np.logical_and(imgarray >= imgscale[i], imgarray < imgscale[i+1]), imgarray, copy=False).mask
    return newimg
    #return [[rebin_func(val, imgscale, truescale) for val in x] for x in imgarray]


def rebin(bins, name="lestermadeline_BW_5mile.jpg"):
    img = Image.open(name)
    
    new_img = rebin_img(img, bins)
    
    %matplotlib notebook
    fig = plt.figure()

    imshow(new_img, cmap='gray')
    plt.axis('off')
    
    plt.show()
    img.close()
    return 

# Rebinning methods (RGB)

Note that it does use the rebin_img function above for each band separately

In [ ]:
def array2Image(array):
    return Image.fromarray(np.uint8(array))

def rebin_rgb_func(img, nbins):
    r,g,b = img.split()
    rnew = array2Image(rebin_img(r, nbins))
    gnew = array2Image(rebin_img(g, nbins))
    bnew = array2Image(rebin_img(b, nbins))
    for band in [rnew, gnew, bnew]:
        bandarray = np.asarray(band)
        #print(np.unique(bandarray))
    return Image.merge("RGB", (rnew, gnew, bnew))
    
def rebin_RGB(bins, name="monterey.png"):
    img = Image.open(name)
    
    new_img = rebin_rgb_func(img, bins)
    
    %matplotlib notebook
    fig = plt.figure()
    imshow(new_img)
    plt.axis('off')
    plt.show()
    img.close()
    return 
